In [ ]:
# AWS Bedrock Agent Demo

This notebook demonstrates the migrated agent using AWS Bedrock Converse API instead of the Anthropic API.

## Features:
- Uses AWS Bedrock Converse API
- Supports Claude 3.7/4 models with reasoning capabilities
- Text, image, and document input support
- **Chat functionality** with ongoing conversations and context memory
- Document upload: txt, md, json, csv, xml, yaml files (up to 5 files, 4.5MB each)
- Mixed content analysis (images + documents together)
- Interactive terminal chat with file upload commands
- Chat history management and export
- No tools/MCP dependencies

## Prerequisites:
- AWS credentials configured (via AWS CLI, environment variables, or IAM role)
- Access to Bedrock models in your AWS account
- boto3 package installed


In [ ]:
import os
import sys
import nest_asyncio
nest_asyncio.apply()

# Add parent directory to path
parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)

from agents.agent import Agent, ModelConfig


In [ ]:
## 1. Basic Text Conversation with Reasoning


In [ ]:
# Create agent with Claude 3.7 Sonnet and reasoning enabled
agent = Agent(
    name="Bedrock Agent",
    system="You are a helpful AI assistant powered by AWS Bedrock. Use step-by-step reasoning for complex problems.",
    config=ModelConfig(
        model="anthropic.claude-3-7-sonnet-20250219-v1:0",  # Claude 3.7 Sonnet
        max_tokens=4096,
        temperature=0.7,
        enable_reasoning=True,  # Enable reasoning capabilities
        reasoning_budget_tokens=2000  # Budget for reasoning process
    ),
    region="us-east-1",  # Change to your preferred region
    verbose=True,
    show_reasoning=True  # Show reasoning output in verbose mode
)

print("Agent with reasoning capabilities initialized successfully!")


In [ ]:
# Complex reasoning problem
response = agent.run("""
A company has 150 employees across 3 departments:
- Engineering: 60 employees earning $95k average
- Sales: 50 employees earning $75k average  
- Marketing: 40 employees earning $65k average

They want to give a 12% raise to all employees, but their budget allows for only $1.8M total salary after raises.
Can they afford this raise? Show your detailed calculations and reasoning.
""")

# The reasoning and final answer will be displayed above due to verbose=True and show_reasoning=True


In [ ]:
## 1.1. Programmatic Access to Reasoning

You can also access reasoning and final response separately for programmatic use:


In [ ]:
# Create a quiet agent (no verbose output) for programmatic access
quiet_agent = Agent(
    name="Quiet Agent",
    system="You are a helpful AI assistant. Think step-by-step through problems.",
    config=ModelConfig(
        model="anthropic.claude-3-7-sonnet-20250219-v1:0",
        enable_reasoning=True,
        reasoning_budget_tokens=1500
    ),
    verbose=False  # No automatic output
)

# Get reasoning and response separately
reasoning, final_response = quiet_agent.get_reasoning_and_response(
    "If I invest $10,000 at 7% annual interest compounded monthly, how much will I have after 5 years?"
)

print("=== REASONING PROCESS ===")
print(reasoning if reasoning else "No reasoning provided")
print("\n=== FINAL ANSWER ===") 
print(final_response)

# Or get them individually
print("\n=== USING INDIVIDUAL METHODS ===")
reasoning_only = quiet_agent.get_reasoning_only("What's the square root of 2 to 10 decimal places?")
response_only = quiet_agent.get_response_only("What's the square root of 2 to 10 decimal places?")

print(f"Reasoning only: {reasoning_only[:100] if reasoning_only else 'None'}...")
print(f"Response only: {response_only}")


In [ ]:
## 2. Image Analysis with Reasoning

Now let's demonstrate the image analysis capabilities combined with reasoning:


In [ ]:
# Create a simple test image for demonstration
import matplotlib.pyplot as plt
import numpy as np

# Create a sample chart
fig, ax = plt.subplots(figsize=(8, 6))
x = np.linspace(0, 10, 100)
y = np.sin(x) * np.exp(-x/10)
ax.plot(x, y, 'b-', linewidth=2, label='Damped Sine Wave')
ax.set_xlabel('Time')
ax.set_ylabel('Amplitude')
ax.set_title('Sample Data Visualization')
ax.legend()
ax.grid(True, alpha=0.3)

# Save the plot
image_path = "sample_chart.png"
plt.savefig(image_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"Sample chart saved as: {image_path}")


In [ ]:
# Analyze the image with reasoning
image_response = agent.run(
    "Please analyze this chart step by step. What type of mathematical function does this appear to represent, and what are the key characteristics?",
    image_paths=[image_path]
)

# The reasoning and analysis will be displayed above due to verbose=True and show_reasoning=True


In [ ]:
## 3. Document Analysis with Reasoning

Now let's demonstrate document upload and analysis capabilities:


In [ ]:
# Create sample documents for testing
import json

# Create a JSON data file
data = {
    "employees": [
        {"name": "Alice", "department": "Engineering", "salary": 95000, "experience": 5},
        {"name": "Bob", "department": "Sales", "salary": 75000, "experience": 3},
        {"name": "Carol", "department": "Marketing", "salary": 65000, "experience": 2},
        {"name": "David", "department": "Engineering", "salary": 105000, "experience": 8}
    ],
    "company_budget": 1800000,
    "proposed_raise": 0.12
}

json_file = "company_data.json"
with open(json_file, "w") as f:
    json.dump(data, f, indent=2)

# Create a markdown report
md_content = """# Company Analysis Report

## Current Situation
Our company has been experiencing growth and we're considering salary adjustments.

## Key Metrics
- Total employees: 150
- Departments: Engineering, Sales, Marketing
- Average tenure: 4.2 years
- Current annual budget: $1.8M

## Recommendations
1. Conduct salary analysis by department
2. Consider performance-based raises
3. Review market rates for competitive positioning

## Next Steps
- Review individual performance
- Calculate budget impact
- Present proposal to board
"""

md_file = "analysis_report.md"
with open(md_file, "w") as f:
    f.write(md_content)

print(f"Created sample documents: {json_file}, {md_file}")

# Analyze the documents with reasoning
doc_response = agent.run(
    "Please analyze these documents and provide recommendations for the salary raise proposal. Use step-by-step reasoning.",
    document_paths=[json_file, md_file]
)

# The reasoning and analysis will be displayed above due to verbose=True and show_reasoning=True


In [ ]:
## 3.1. Document Convenience Methods


In [ ]:
# Document convenience methods
print("=== Single Document Analysis ===")
single_doc_response = agent.add_document_from_file(
    json_file,
    "Summarize the key information in this employee data file"
)
print("Single document analysis completed.\n")

print("=== Multiple Documents Analysis ===")
multi_doc_response = agent.add_documents_from_files(
    [json_file, md_file],
    "Compare the data in these files and identify any discrepancies"
)
print("Multiple documents analysis completed.\n")

print("=== Mixed Content: Image + Documents ===")
mixed_response = agent.add_mixed_files(
    "Analyze this chart and the company documents together. How does the mathematical pattern relate to business planning?",
    image_paths=[image_path],
    document_paths=[json_file]
)
print("Mixed content analysis completed.")


In [ ]:
## 4. Chat Functionality

Now let's demonstrate the chat capabilities for ongoing conversations:


In [ ]:
# Create a chat agent (less verbose for cleaner conversation)
chat_agent = Agent(
    name="Chat Agent",
    system="You are a helpful AI assistant. Be conversational and remember our discussion context.",
    config=ModelConfig(
        model="anthropic.claude-3-7-sonnet-20250219-v1:0",
        enable_reasoning=True,
        reasoning_budget_tokens=1000  # Lower budget for faster chat
    ),
    verbose=False  # Less verbose for cleaner chat
)

print("🤖 Chat agent created! Let's have a conversation...")

# Start a conversation
response1 = chat_agent.chat("Hi! I'm working on a Python project about data analysis.")
print("🤖:", response1["output"]["message"]["content"][0]["text"])

# Continue the conversation - the agent remembers context
response2 = chat_agent.chat("What are some good libraries I should consider?")
print("\n🤖:", response2["output"]["message"]["content"][0]["text"])

# Continue with more specific question
response3 = chat_agent.chat("I'm specifically working with financial data. Which of those would be best?")
print("\n🤖:", response3["output"]["message"]["content"][0]["text"])


In [ ]:
# Continue the chat by adding files
# First, let's reuse our previous files
print("\n📄 Adding a document to the ongoing conversation...")

# The agent remembers the context about financial data analysis
response4 = chat_agent.chat_with_files(
    "Here's some sample data I'm working with. Can you help me analyze it in the context of our discussion?",
    document_paths=[json_file]
)
print("🤖:", response4["output"]["message"]["content"][0]["text"])

# Add an image to the same conversation
print("\n🖼️ Adding an image to the same conversation...")
response5 = chat_agent.chat_with_files(
    "I also have this chart. How does it relate to the data file we just discussed?",
    image_paths=[image_path]
)
print("🤖:", response5["output"]["message"]["content"][0]["text"])


In [ ]:
# Chat history management
print("\n📜 Chat History Features:")
print(f"Number of messages in conversation: {len(chat_agent.get_chat_history())}")

# Print the chat history
chat_agent.print_chat_history()

# Export chat history
history_file = chat_agent.export_chat_history("demo_chat_history.json")

print(f"\n🔄 You can also clear the history and start fresh:")
print("chat_agent.clear_chat_history()")

print(f"\n💡 For interactive terminal chat, use:")
print("chat_agent.start_interactive_chat()")
print("(This would start an interactive session with commands like /image, /doc, /history, etc.)")


In [ ]:
## 5. Cleanup


In [ ]:
# Clean up demo files
import os

files_to_clean = [image_path, json_file, md_file, "demo_chat_history.json"]

for file_path in files_to_clean:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Removed: {file_path}")

print("\nDemo completed successfully! All test files cleaned up.")
